In [27]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import time
import json
from datetime import date
from dateutil.parser import parse
import pandas as pd
import xml.etree.ElementTree as ET

In [34]:
url = 'http://oms.coastalemc.com/OMSWebMap/OMSWebMap.htm?clientKey=undefined' #Coastal Electric Cooperation
chrome_driver_path = '/Users/xuanedx1/Desktop/URG/web-scaper/chromedriver'

In [35]:
def get_logs(url):

    desired_capabilities = DesiredCapabilities.CHROME
    desired_capabilities["goog:loggingPrefs"] = {"performance": "ALL"}

    # Create the webdriver object and pass the arguments
    options = webdriver.ChromeOptions()

    # Chrome will start in Headless mode
    options.add_argument('headless')

    # Ignores any certificate errors if there is any
    options.add_argument("--ignore-certificate-errors")

    # Startup the chrome webdriver with executable path and
    # pass the chrome options and desired capabilities as
    # parameters.
    driver = webdriver.Chrome(executable_path=chrome_driver_path,
                              chrome_options=options,
                              desired_capabilities=desired_capabilities)

    # Send a request to the website and let it load
    driver.get(url)

    # Sleeps for 5 seconds
    time.sleep(5)

    logs_raw = driver.get_log("performance")
    logs = [json.loads(lr["message"])["message"] for lr in logs_raw]
    
    return driver, logs


In [36]:
def summary_filter(log_):
    return (
        # is an actual response
        log_["method"] == "Network.responseReceived"
        # and json
        and "xml" in log_["params"]["response"]["mimeType"]
        and "GetStagingInfo" in log_["params"]["response"]["url"]
    )

def report_filter(log_):
    return (
        # is an actual response
        log_["method"] == "Network.responseReceived"
        # and json
        and "xml" in log_["params"]["response"]["mimeType"]
        and "GetAllCounties" in log_["params"]["response"]["url"]
    )

In [37]:
def get_layout_3_df(url):
    driver, logs = get_logs(url)
    
    summary_response = ""
    for log in filter(summary_filter, logs):
        request_id = log["params"]["requestId"]
        resp_url = log["params"]["response"]["url"]
        print(f"Caught {resp_url}")
        summary_response = driver.execute_cdp_cmd("Network.getResponseBody", {"requestId": request_id})['body']
        
    tree = ET.ElementTree(ET.fromstring(summary_response))
    root = tree.getroot()
    timestamp = date.today().strftime("%B %d, %Y") + " " + root[0].text
    
    report_response = ""
    for log in filter(report_filter, logs):
        request_id = log["params"]["requestId"]
        resp_url = log["params"]["response"]["url"]
        print(f"Caught {resp_url}")
        report_response = driver.execute_cdp_cmd("Network.getResponseBody", {"requestId": request_id})['body']
    
    layout_3_df = pd.read_xml(report_response)
    layout_3_df.insert(loc=0, column='timestamp', value=timestamp)
    
    return layout_3_df

In [38]:
get_layout_3_df(url)

/var/folders/3j/7cy055ys1yz5dcj9cwbxbw8h0000gp/T/ipykernel_55148/3593499440.py:18: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=chrome_driver_path,
/var/folders/3j/7cy055ys1yz5dcj9cwbxbw8h0000gp/T/ipykernel_55148/3593499440.py:18: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path=chrome_driver_path,


ParseError: no element found: line 1, column 0 (<string>)

In [7]:
get_layout_3_df("http://map.irwinemc.com/omswebmap/OMSWebMap.htm?clientKey=undefined") #Irwin EMC

C:\Users\saira\AppData\Local\Temp\ipykernel_4292\1350414708.py:18: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=chrome_driver_path,
C:\Users\saira\AppData\Local\Temp\ipykernel_4292\1350414708.py:18: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path=chrome_driver_path,


Caught http://map.irwinemc.com/omswebmap/MobileMap/OMSMobileService.asmx/GetStagingInfo
Caught http://map.irwinemc.com/omswebmap/MobileMap/OMSMobileService.asmx/GetAllCounties
Caught http://map.irwinemc.com/omswebmap/MobileMap/OMSMobileService.asmx/GetAllCounties


,timestamp,CountyName,CustomersServed,CustomersAffected,Shape
0,"November 03, 2022 03:48 PM",Worth,47,0,"-83.666,31.468;-83.666,31.458;-83.666,31.452;-..."
1,"November 03, 2022 03:48 PM",Coffee,106,0,"-83.135,31.473;-83.134,31.473;-83.13,31.474;-8..."
2,"November 03, 2022 03:48 PM",Wilcox,396,0,"-83.226,31.845;-83.242,31.845;-83.244,31.845;-..."
3,"November 03, 2022 03:48 PM",Berrien,568,0,"-83.165,31.127;-83.165,31.127;-83.165,31.127;-..."
4,"November 03, 2022 03:48 PM",Tift,635,0,"-83.37,31.436;-83.37,31.423;-83.37,31.418;-83...."
5,"November 03, 2022 03:48 PM",Turner,2553,1,"-83.459,31.725;-83.458,31.724;-83.458,31.723;-..."
6,"November 03, 2022 03:48 PM",Ben Hill,3306,1,"-83.325,31.747;-83.325,31.748;-83.325,31.75;-8..."
7,"November 03, 2022 03:48 PM",Irwin,4936,0,"-83.339,31.476;-83.339,31.476;-83.341,31.476;-..."
